In [1]:
!pip install map_boxes
!pip install tqdm
!pip install pycocotools

In [2]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [3]:
GT_JSON = '/opt/ml/detection/dataset/train.json'
PRED_CSV = '/opt/ml/detection/faster_rcnn/submission_ensemble_pseudoLabeling_3_0.6.csv'


    
# load ground truth
with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)

   

In [4]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2 confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''
    
new_pred = []

file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()

print(file_names[1])
print(bboxes[1])


train/0001.jpg
0 0.8984344601631165 619.6401977539062 692.5023803710938 689.9927368164062 790.3583374023438 0 0.7132421135902405 422.4284362792969 686.8781127929688 635.5785522460938 855.9871826171875 0 0.6278756856918335 383.41436767578125 678.5245971679688 605.3501586914062 800.9808349609375 4 0.9740090370178223 727.2628784179688 314.5044860839844 992.1361694335938 567.509765625 5 0.8138903975486755 63.92866897583008 447.2177734375 786.5399780273438 683.38818359375 5 0.7113310098648071 363.8210144042969 670.5466918945312 587.4439697265625 773.8440551757812 7 0.8088732957839966 94.51702117919922 600.0848999023438 230.53250122070312 653.2649536132812 9 0.7919033765792847 20.735858917236328 451.10687255859375 780.5601196289062 698.2651977539062 


In [5]:


'''
create new_pred
'''
    
for i, bbox in enumerate(bboxes):
    if isinstance(bbox, float):
        print(f'{file_names[i]} empty box')

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).split(' '))
    
    if len(boxes) % 6 == 1:
        boxes = boxes[:-1].reshape(-1, 6)
    elif len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    else:
        raise Exception('error', 'invalid box count')
    for box in boxes:
        new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])

train/4432.jpg empty box


4883it [00:00, 33380.52it/s]


In [6]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2, confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''
    
gt = []

'''
create gt
'''
    
coco = COCO(GT_JSON)
   
'''
coco.getImgIds(): return image id list
    
coco.loadImgs(image_id): return image_info
    
image_info['file_name']: return file name
   
coco.getAnnIds(imgIds=image_info['id']): return annotation id
    
coco.loadAnns(ann_ids): return annotation information list (annotation_info_list)
    
annotation_info_list[i]['bbox']: return i'th annotation [x_min, y_min, w, h]
    
annotation_info_list[i]['category_id']: return i'th annotation category
    
'''
    
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                   float(annotation['bbox'][0]),
                   float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                   float(annotation['bbox'][1]),
                   (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [7]:
'''
calculate mAP
'''


mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5)

print(mean_ap)

Number of files in annotations: 4883
Number of files in predictions: 4882
Unique classes: 10
Detections length: 4882
Annotations length: 4883
0                              | 0.649690 |    3966
1                              | 0.650620 |    6352
2                              | 0.766833 |     897
3                              | 0.747924 |     936
4                              | 0.807634 |     982
5                              | 0.668932 |    2943
6                              | 0.685864 |    1263
7                              | 0.814463 |    5178
8                              | 0.923432 |     159
9                              | 0.832812 |     468
mAP: 0.754820
0.7548203867789997
